In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Run hyperparameter tuning for a TensorFlow model

<table align="left">

  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Ftraining%2Fhyperparameter_tuning_tensorflow.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>  
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>                                                                                        
</table>

## Overview

The values you select for your model’s hyperparameters can make all the difference. If you’re only trying to tune a handful of hyperparameters, you might be able to run experiments manually. But when you start juggling hyperparameters for your model’s architecture, the optimizer, and finding the best batch size and learning rate, automating these experiments at scale quickly becomes a necessity. 

And it’s not just about tracking the results from all these trials. You also want a way to efficiently search the space of possible values so you don’t waste as much time trying out combinations that yield low accuracy scores.

Vertex AI Training includes a hyperparameter tuning service. A Vertex AI Hyperparameter tuning job will run multiple trials of your training code. On each trial, it will use different values for your chosen hyperparameters, set within limits you specify. By default, the service uses Bayesian optimization to search the space of possible hyperparameter values. This means that information from prior experiments is used to select the next set of values, making the search more efficient. 

Learn more about [Vertex AI Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview).

### Objective

In this tutorial, you learn how to run a Vertex AI Hyperparameter Tuning job for a TensorFlow model. While this example uses TensorFlow, you can also use this service for other ML frameworks.

This tutorial uses the following Google Cloud ML services and resources:

* Vertex AI Training
* Cloud Storage
* Artifact Registry


The steps performed include:

* Modify training application code for automated hyperparameter tuning.
* Containerize training application code.
* Configure and launch a hyperparameter tuning job with the Vertex AI Python SDK.

### Dataset

This sample uses the [Horses or Humans dataset](https://www.tensorflow.org/datasets/catalog/horses_or_humans) available through [TensorFlow datasets](https://www.tensorflow.org/datasets) to train a binary image classification model (horse or human).

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
[Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing)
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK and other required packages

In [5]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [6]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [15]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique2"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [16]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://your-bucket-name-vertexai-service-project-unique2/...


To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [6]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}

import os

import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Containerize training application code

Before you can run a hyperparameter tuning job, you must create a source code file (training script) and a Dockerfile.

The source code trains a model in the ML framework of your choice. In this example, you'll use TensorFlow to train a classification model.

The Dockerfile will include all the commands needed to run the image. It'll install all the libraries required by your training script, and set up the entry point for the training code. 

You'll create a directory for storing your source code file and Dockerfile.

In [19]:
APPLICATION_DIR = "hptune"
TRAINER_DIR = f"{APPLICATION_DIR}/trainer"

In [20]:
!mkdir -p $APPLICATION_DIR
!mkdir -p $TRAINER_DIR

### Write the training script

In [21]:
%%writefile {TRAINER_DIR}/task.py

import tensorflow as tf
import tensorflow_datasets as tfds
import argparse
import hypertune

NUM_EPOCHS = 10


def get_args():
  '''Parses args. Must include all hyperparameters you want to tune.'''

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate',
      required=True,
      type=float,
      help='learning rate')
  parser.add_argument(
      '--momentum',
      required=True,
      type=float,
      help='SGD momentum value')
  parser.add_argument(
      '--num_units',
      required=True,
      type=int,
      help='number of units in last hidden layer')
  args = parser.parse_args()
  return args


def preprocess_data(image, label):
  '''Resizes and scales images.'''

  image = tf.image.resize(image, (150,150))
  return tf.cast(image, tf.float32) / 255., label


def create_dataset():
  '''Loads Horses Or Humans dataset and preprocesses data.'''

  data, info = tfds.load(name='horses_or_humans', as_supervised=True, with_info=True)

  # Create train dataset
  train_data = data['train'].map(preprocess_data)
  train_data  = train_data.shuffle(1000)
  train_data  = train_data.batch(64)

  # Create validation dataset
  validation_data = data['test'].map(preprocess_data)
  validation_data  = validation_data.batch(64)

  return train_data, validation_data


def create_model(num_units, learning_rate, momentum):
  '''Defines and compiles model.'''

  inputs = tf.keras.Input(shape=(150, 150, 3))
  x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(inputs, outputs)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum),
      metrics=['accuracy'])
  return model


def main():
  args = get_args()
  train_data, validation_data = create_dataset()
  model = create_model(args.num_units, args.learning_rate, args.momentum)
  history = model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data)

  # DEFINE METRIC
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=NUM_EPOCHS)


if __name__ == "__main__":
    main()

Writing hptune/trainer/task.py


### Understanding the training script

Before you build the container, let's take a deeper look at the code. There are a few components that are specific to using the hyperparameter tuning service.

The script imports the **hypertune library**:

`import hypertune`

The **function `get_args()`** defines a command-line argument for each hyperparameter that you want to tune. In this example, the hyperparameters that will be tuned are the learning rate, the momentum value in the optimizer, and the number of units in the last hidden layer of the model. Feel free to experiment with others. The values passed in those arguments are then used to set the corresponding hyperparameter in the code.

```
def get_args():
  '''Parses args. Must include all hyperparameters you want to tune.'''

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate',
      required=True,
      type=float,
      help='learning rate')
  parser.add_argument(
      '--momentum',
      required=True,
      type=float,
      help='SGD momentum value')
  parser.add_argument(
      '--num_units',
      required=True,
      type=int,
      help='number of units in last hidden layer')
  args = parser.parse_args()
  return args
```


At the end of the `main()` function, the `hypertune` library is used to **define the metric you want to optimize**. In TensorFlow, the keras `model.fit` method returns a `History` object. The `History.history` attribute is a record of training loss values and metrics values at successive epochs. If you pass validation data to `model.fit` the `History.history` attribute will include validation loss and metrics values as well.

```
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=NUM_EPOCHS)
 ```
 
For example, if you trained a model for three epochs with validation data and provided accuracy as a metric, the `History.history` attribute would look similar to the following dictionary.

```
{
 "accuracy": [
   0.7795261740684509,
   0.9471358060836792,
   0.9870933294296265
 ],
 "loss": [
   0.6340447664260864,
   0.16712145507335663,
   0.04546636343002319
 ],
 "val_accuracy": [
   0.3795261740684509,
   0.4471358060836792,
   0.4870933294296265
 ],
 "val_loss": [
   2.044623374938965,
   4.100203514099121,
   3.0728273391723633
 ]
```

If you want the hyperparameter tuning service to discover the values that maximize the model's validation accuracy, you define the metric as the last entry (or `NUM_EPOCS` - 1) of the `val_accuracy` list. Then, pass this metric to an instance of `HyperTune`. You can pick whatever string you like for the `hyperparameter_metric_tag`, but you need to use the string again later when you kick off the hyperparameter tuning job.

### Write Dockerfile

After writing your training code, you create a Dockerfile. In the Dockerfile, you include all the commands needed to run the image. It'll install all the necessary libraries, including the CloudML Hypertune library, and set up the entry point for the training code.

In [22]:
%%writefile {APPLICATION_DIR}/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8

WORKDIR /

# Installs hypertune library
RUN pip install cloudml-hypertune

# Copies the trainer code to the Docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Writing hptune/Dockerfile


### Build the container

You'll store the Docker image in Artifact Registry. First, create a Docker repository in Artifact Registry

In [23]:
REPO_NAME='horses-app'

! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

!gcloud artifacts repositories create $REPO_NAME --repository-format=docker \
--location=$REGION --description="Docker repository"

! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

Create request issued for: [horses-app]
Waiting for operation [projects/vertexai-service-project/locations/us-central1/
operations/21090343-a309-48dd-8cad-845b94da4ce0] to complete...done.           
Created repository [horses-app].

{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
Docker configuration file updated.


Define a variable with the URI of your Docker image in Artifact Registry:

In [24]:
IMAGE_URI = (
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/horse_human_hptune:latest"
)

Then, build the container by running the following:

In [25]:
cd $APPLICATION_DIR

/home/jupyter/vertex-ai-samples/notebooks/official/training/hptune


In [26]:
! docker build ./ -t $IMAGE_URI

Sending build context to Docker daemon  10.24kB
Step 1/5 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8
latest: Pulling from deeplearning-platform-release/tf2-gpu.2-8

4c3075c9: Pulling fs layer 
5ac03608: Pulling fs layer 
a0f44e37: Pulling fs layer 
897adb41: Pulling fs layer 
2bd2f1fc: Pulling fs layer 
48468c4c: Pulling fs layer 
62c9634f: Pulling fs layer 
73afe479: Pulling fs layer 
90fac1fd: Pulling fs layer 
12d52855: Pulling fs layer 
42cd5e49: Pulling fs layer 
e42b0909: Pulling fs layer 
7530d9d6: Pulling fs layer 
794f0746: Pulling fs layer 
b700ef54: Pulling fs layer 
8d20e619: Pulling fs layer 
90fac1fd: Waiting fs layer 
a87488c4: Pulling fs layer 
b0a15b06: Pulling fs layer 
b1eed34b: Pulling fs layer 
e2394a74: Pulling fs layer 
108e4f90: Pulling fs layer 
afb11616: Pulling fs layer 
3e0481a4: Pulling fs layer 
603905d5: Pulling fs layer 
d342008b: Pulling fs layer 
9da14003: Pulling fs layer 
e680d75e: Pulling fs layer 
292db32a: Pulling fs layer 
50db43a1: P

Finally, push it to Artifact Registry

In [27]:
! docker push $IMAGE_URI

The push refers to repository [us-central1-docker.pkg.dev/vertexai-service-project/horses-app/horse_human_hptune]

33cbe2f4: Preparing 
4a1034eb: Preparing 
decf686b: Preparing 
b26c5aa6: Preparing 
ab8b6e25: Preparing 
8c778906: Preparing 
bf18a086: Preparing 
8f9c0e18: Preparing 
727d3bb3: Preparing 
34c1abe4: Preparing 
5bbe2416: Preparing 
fdd37854: Preparing 
32853325: Preparing 
3aa80617: Preparing 
0cb7de16: Preparing 
aafacb94: Preparing 
fc6acdda: Preparing 
67e071e3: Preparing 
c4c31754: Preparing 
ad20f466: Preparing 
cdd9dd8d: Preparing 
75ffbe66: Preparing 
3727400b: Preparing 
0a8273de: Preparing 
5ae6a810: Preparing 
40c1c7de: Preparing 
c800ebdc: Preparing 
40dc4164: Preparing 
f18a086: Preparing 
f72e3446: Preparing 
311fde8e: Preparing 
f5722498: Preparing 
2c163a83: Preparing 
13d8681b: Preparing 
482cf441: Preparing 
ba9dc290: Preparing 
a8f43981: Preparing 
f5c71bf6: Preparing 
4a4d2337: Preparing 
30c0e62e: Preparing 
da122002: Preparing 
34842944: Preparing 
f9c0

## Configure a hyperparameter tuning job

Now that your training application code is containerized, it's time to specify and run the hyperparameter tuning job.

To launch the hyperparameter tuning job, you need to first define the `worker_pool_specs`, which specifies the machine type and Docker image. The following spec defines one `n1-standard-4` machine with one `NVIDIA Tesla T4` GPU as the accelerator.

In [28]:
# The spec of the worker pools including machine type and Docker image
# Be sure to replace PROJECT_ID in the `image_uri` with your project.

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/horse_human_hptune:latest"
        },
    }
]

### Define parameter spec

Next, define the `parameter_spec`, which is a dictionary specifying the parameters you want to optimize. The **dictionary key** is the string you assigned to the command line argument for each hyperparameter, and the **dictionary value** is the parameter specification.

For each hyperparameter, you need to define the `Type` as well as the bounds for the values that the tuning service will try. Hyperparameters can be of type `Double`, `Integer`, `Categorical`, or `Discrete`. If you select the type `Double` or `Integer`, you need to provide a minimum and maximum value. And if you select `Categorical` or `Discrete` you need to provide the values. For the `Double` and `Integer` types, you also need to provide the scaling value. Learn more about [Using an Appropriate Scale](https://www.youtube.com/watch?v=cSoK_6Rkbfg).

In [29]:
# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "num_units": hpt.DiscreteParameterSpec(values=[64, 128, 512], scale=None),
}

The final spec to define is `metric_spec`, which is a dictionary representing the metric to optimize. The dictionary key is the `hyperparameter_metric_tag` that you set in your training application code, and the value is the optimization goal.

In [30]:
# Dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec = {"accuracy": "maximize"}

Once the specs are defined, you create a `CustomJob`, which is the common spec that will be used to run your job on each of the hyperparameter tuning trials.

In [31]:
my_custom_job = aiplatform.CustomJob(
    display_name="horses-humans-sdk-job",
    worker_pool_specs=worker_pool_specs,
    staging_bucket=BUCKET_URI,
)

Then, create and run `HyperparameterTuningJob`.

In [32]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name="horses-humans-sdk-job",
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=10,
    parallel_trial_count=3,
)

hp_job.run()

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/314481442207/locations/us-central1/hyperparameterTuningJobs/2567320377676005376
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/314481442207/locations/us-central1/hyperparameterTuningJobs/2567320377676005376')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2567320377676005376?project=314481442207
HyperparameterTuningJob projects/314481442207/locations/us-central1/hyperparameterTuningJobs/2567320377676005376 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/314481442207/locations/us-central1/hyperparameterTuningJobs/2567320377676005376 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/314481442207/locations/us-central1/hyperparameterTuningJobs/2567320377676005376 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJ

There are a few arguments to note:


* **max_trial_count**: You need to put an upper bound on the number of trials the service will run. More trials generally leads to better results, but there will be a point of diminishing returns, after which additional trials have little or no effect on the metric you're trying to optimize. It is a best practice to start with a smaller number of trials and get a sense of how impactful your chosen hyperparameters are before scaling up.

* **parallel_trial_count**: If you use parallel trials, the service provisions multiple training processing clusters. Increasing the number of parallel trials reduces the amount of time the hyperparameter tuning job takes to run; however, it can reduce the effectiveness of the job overall. This is because the default tuning strategy uses results of previous trials to inform the assignment of values in subsequent trials.

* **search_algorithm**: You can set the search algorithm to grid, random, or default (None). If you do not specify an algorithm, as shown in this example, the default option applies Bayesian optimization to search the space of possible hyperparameter values and is the recommended algorithm. Learn more about [Hyperparameter tuning using Bayesian Optimization](https://cloud.google.com/blog/products/ai-machine-learning/hyperparameter-tuning-cloud-machine-learning-engine-using-bayesian-optimization)

## Examine results

Click on the generated link in the output to see your run in the Cloud Console. When the job completes, you will see the results of the tuning trials.

You can sort the results by the optimization metric and then set the hyperparamers in your training application code to the values from the trial with the highest accuracy.

![console_ui_results](tuning_results.png)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [33]:
# Delete artifact registry repo
! gcloud artifacts repositories delete $REPO_NAME --location $REGION --quiet

delete_custom_job = True
delete_bucket = False

# Delete hptune job
if delete_custom_job:
    try:
        hp_job.delete()
    except Exception as e:
        print(e)

# Delete bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

Delete request issued for: [horses-app]
Waiting for operation [projects/vertexai-service-project/locations/us-central1/
operations/d908e713-633c-44f4-b3e2-9e1426f0a909] to complete...done.           
Deleted repository [horses-app].
Deleting HyperparameterTuningJob : projects/314481442207/locations/us-central1/hyperparameterTuningJobs/2567320377676005376
Delete HyperparameterTuningJob  backing LRO: projects/314481442207/locations/us-central1/operations/5246742253635895296
HyperparameterTuningJob deleted. . Resource name: projects/314481442207/locations/us-central1/hyperparameterTuningJobs/2567320377676005376
